#PAINEL COVID GYN


```
painel-covid-goiania.herokuapp.com
github.com/wendelmarques/painel-covid-goiania
```



### Objetivo: Criar vizualizações com os dados sobre os casos confirmados da COVID-19 em Goiânia.

##### Os dados utilizados para geração do mapa e dos gráficos, foram retirados dos Informes Epidemiológicos publicados (em PDFs) pela Prefeitura de Goiânia, por meio de uma técnica chamada Data Scraping (algoritmos que realizam a tarefa de extração). São divulgadas apenas informações sobre alguns bairros - os que possuem mais casos confirmados acumulados, por isso, foram extraídos dados de 111 bairros.

#####Para mais informações, acesse: saude.goiania.go.gov.br


<br>
<br>

#### Fontes/ créditos

*    [Brasil.IO - O Brasil em dados libertos](https://brasil.io/dataset/covid19/caso/?search=&date=&state=&city=&place_type=&is_last=&city_ibge_code=5208707&order_for_place=)
*    [Plotly - Repositório](https://github.com/plotly/dash-sample-apps/tree/master/apps)
*   [How to create outstanding animated scatter maps with Plotly and Dash (Medium) - Lamothe Thibaud ](https://towardsdatascience.com/how-to-create-animated-scatter-maps-with-plotly-and-dash-f10bb82d357a)
*    [Deploying Dash or Flask web application on Heroku. Easy CI/CD (Medium) - Lamothe Thibaud](https://towardsdatascience.com/deploying-dash-or-flask-web-application-on-heroku-easy-ci-cd-4111da3170b8)
*    [Official Release of bar_chart_race (Medium) - Ted Petrou
dexplo.org/bar_chart_race](https://medium.com/dunder-data/bar-chart-race-python-package-official-release-78a420e182a2)
*    [Gráfico de Corrida de Barras | Dica Pandas #7 - Programação Dinâmica (YouTube)](https://www.youtube.com/watch?v=rIwxjCnvdcY)
*    [Como utilizar a Google Geocoding API para obter endereços](https://www.devmedia.com.br/como-utilizar-a-google-geocoding-api-para-obter-enderecos/36751)
*    [Python Client for Google Maps Services](https://github.com/googlemaps/google-maps-services-python)
* [Como fazer um Web Scraping com Python](https://goomore.com/blog/web-scraping-python/)

* [Official Release of bar_chart_race](https://medium.com/dunder-data/bar-chart-race-python-package-official-release-78a420e182a2)

* [Corrida de CASOS de COVID no Brasil | Gráfico de Corrida de Barras (Bar Chart Race) | Dica Pandas #7](https://www.youtube.com/watch?v=rIwxjCnvdcY)

* [ACESSANDO RECURSOS NA WEB COM PYTHON](https://pythonhelp.wordpress.com/2013/03/12/acessando-recursos-na-web-com-python/)

* [Como usar o R para escolher um lugar para morar (3) - Converter CEP em coordenadas geográficas](https://sillasgonzaga.github.io/2016-11-18-olx3/#:~:text=Obter%20endere%C3%A7o%20a%20partir%20do%20CEP&text=Sabemos%20que%20quanto%20mais%20dados,%2C%20bairro%2C%20cidade%20e%20estado.&text=%C3%89%20necess%C3%A1rio%20juntar%20todas%20as,de%20endere%C3%A7os%20em%20uma%20s%C3%B3)





---



Criado por Wendel Marques

*   [GitHub](https://www.github.com/WendelMarques)
*   [LinkedIn](https://www.linkedin.com/in/wendelmarques/)
*   [Medium](https://medium.com/@WendelMarques)


# Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Biblioteca e pacotes

In [ ]:
import pandas as pd
import time
import bs4
import requests
from datetime import datetime
!pip install tika
from tika import parser
import numpy as np
import urllib.request #biblioteca usada para consultar uma URL
from bs4 import BeautifulSoup #funções BeautifulSoup para analisar os dados retornados do site
!pip install googlemaps
import googlemaps
import math
import numpy as np
!pip install bar_chart_race
!pip install ffmpeg
import bar_chart_race as bcr
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px

# #https://medium.com/dunder-data/bar-chart-race-python-package-official-release-78a420e182a2

In [ ]:
#para salvar os graficos em png e svg
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
from google.colab import files


# Etapas a partir da extração do Informe Epidemiológico
`Extração de dados dos Informes Epidemiológicos (PDFs) publicados no site da Prefeitura de Goiânia`



*   Bar race (por bairro e por região)
*   Mapa






## Data scraping - total de casos por bairro

In [ ]:
#acessa a pagina de informes
siteInformes = "https://saude.goiania.go.gov.br/goiania-contra-o-coronavirus/informe-epidemiologico-covid-19/#antigos"
#Consulta o site e retorne o html para a variável 'page'
page = urllib.request.urlopen(siteInformes)
#Parse o html na variável 'page' e armazene-o no formato BeautifulSoup
soup = BeautifulSoup(page, 'html5lib')

In [ ]:
#pecorre cada linha do html de siteInformes e pegar os links dos pdfs
all_links = soup.find_all('a')
urls_pdfs = []
for link in all_links:
    if link.get('href'):
        if link.get('href')[-3:] == 'pdf':
            urls_pdfs.append(link.get('href'))
            
urls_pdfs

['https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-169-18.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-168-17.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-167-16.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-166-15.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-165-14.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-162-11.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-161-10.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-160-09.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-159-08.09.2020.pdf',
 'https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-1

In [ ]:
len(urls_pdfs)

218

In [ ]:
len(np.unique(urls_pdfs)) #deve ser igual ao numero de informes publicados - 2

167

In [ ]:
#remove link duplicados
urls_pdfs = np.unique(urls_pdfs)

In [ ]:
len(urls_pdfs)

167

In [ ]:
#substitui alguns caracteres para evitar erros
urls_pdfs_modificadas = []
for url in urls_pdfs:
    new_link = url.replace('º', '%C2%BA')
    new_link1 = new_link.replace('padrão', 'padra%CC%83o')
    urls_pdfs_modificadas.append(new_link1)


i = 0
for i in range(len(urls_pdfs_modificadas)):
    print(i)
    print(urls_pdfs_modificadas[i])  

0
https://goiania.go.gov.br/sms/wp-uploads/sites/3/2020/05/Informe-COVID-19-n%C2%BA-30-02.05.2020.pdf
1
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-1-03-04-2020.pdf
2
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-10-12.04.2020.pdf
3
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-11-13.04.2020.pdf
4
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-12-14.04.2020.pdf
5
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-13-15.04.2020-2.pdf
6
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-14-16.04.2020.pdf
7
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-15-17.04.2020_2.pdf
8
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-16-18.04.2020.pdf
9
https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/In

In [ ]:
###############################################################
######### OBJETIVO: LER O HTML E EXTRAIR OS DADOS #############
###############################################################

num_informes = len(urls_pdfs) #quantidades de informes publicados
i = 0
j = 0
conteudos = []
dataset = {}

def pega_data(url):
    indice = 0
    while indice < len(url):
        if url[indice] == '2':
            if url[indice+1] == '0':
                if url[indice+2] == '2':
                    if url[indice+3] == '0':
                        if url[indice+4] != '/':
                            data = url[indice] + url[indice+1]+url[indice+2]+url[indice+3]+'-'+url[indice-3]+url[indice-2]+'-'+url[indice-6]+url[indice-5]
                            # print(data)
                            return data
                
        
        indice+=1

    return '0000-00-00'        



def cria_lista_bairros_e_casos(bairro_casos):
    try:
        num_casos = bairro_casos.split(' ')[-1]
        num_casos = int(num_casos)
        qdt_dig_casos = int (math.log10 (num_casos))+2
        bairro = bairro_casos[:len(bairro_casos)-qdt_dig_casos]
        return [bairro, num_casos]

    except ValueError as e:
        return [0, 0]


for url in urls_pdfs_modificadas:

    if i <= num_informes:
        print(i)
        i+=1
        conteudos.clear()
        conteudos.append(parser.from_file(url))
        time.sleep(8)

        #armazena o conteudo do pdf em conteudos_pdfs.txt (um pdf por vez)
        #repare no conteudos.clear() linhas acima
        arquivo = open('conteudos-pdfs.txt', 'w')
        for linha in conteudos:
            arquivo.write(linha['content']) 
        arquivo.close()

        #pega a data do informe
        data = pega_data(url)
        data = data[:10]
        print(data)

        #abre o arquivo e captura a data em que foi criado
        arquivo = open('conteudos-pdfs.txt', 'r')
        lista = arquivo.readlines()
        arquivo.close()

        df = pd.DataFrame(lista)
        delete_row = df[df[0]=='\n'].index
        df = df.drop(delete_row)
        df.reset_index(inplace=True)
        df.drop(columns=['index'], axis=1, inplace=True)

        row_iterator = df.iterrows()

        while 1:
            try:
                for aux in range(len(df)):
                    linha_atual = next(row_iterator)
                    
                    if 'Setor Bueno' in linha_atual[1][0]:
                        while 1:
                            #cria um dicionario no formato bairro: num_casos
                            ret = cria_lista_bairros_e_casos(linha_atual[1][0])
                            if (ret[0] != 0 and ret[0] != 0):
                                dataset[j] = {'data': data, 'bairro': ret[0], 'num_casos': ret[1]}
                                j+=1
                            #"outros" é a flag usada para parar o loop
                            if 'Outros' in linha_atual[1][0]:                        
                                break  
                            linha_atual = next(row_iterator)            

            except StopIteration as e:
                    print(e)
                    break
    else:
        i+=1




2020-09-19 13:58:15,554 [MainThread  ] [INFO ]  Retrieving https://goiania.go.gov.br/sms/wp-uploads/sites/3/2020/05/Informe-COVID-19-n%C2%BA-30-02.05.2020.pdf to /tmp/sms-wp-uploads-sites-3-2020-05-informe-covid-19-n-c2-ba-30-02.05.2020.pdf.


0


2020-09-19 13:58:18,974 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-09-19 13:58:19,937 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-09-19 13:58:20,380 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2020-09-19 13:58:45,171 [MainThread  ] [INFO ]  Retrieving https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-1-03-04-2020.pdf to /tmp/wp-uploads-sites-3-2020-04-informe-covid-19-n-c2-ba-1-03-04-2020.pdf.


2020-05-02

1


2020-09-19 13:58:54,962 [MainThread  ] [INFO ]  Retrieving https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/04/Informe-COVID-19-n%C2%BA-10-12.04.2020.pdf to /tmp/wp-uploads-sites-3-2020-04-informe-covid-19-n-c2-ba-10-12.04.2020.pdf.


...

2020-09-15

164


2020-09-19 14:26:21,322 [MainThread  ] [INFO ]  Retrieving https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-168-17.09.2020.pdf to /tmp/wp-uploads-sites-3-2020-09-informe-covid-19-no-168-17.09.2020.pdf.


2020-09-16

165


2020-09-19 14:26:31,257 [MainThread  ] [INFO ]  Retrieving https://saude.goiania.go.gov.br/wp-uploads/sites/3/2020/09/Informe-COVID-19-no-169-18.09.2020.pdf to /tmp/wp-uploads-sites-3-2020-09-informe-covid-19-no-169-18.09.2020.pdf.


2020-09-17

166
2020-09-18



In [ ]:
dataset

In [ ]:
covid_goiania_por_bairro = pd.DataFrame(dataset)
covid_goiania_por_bairro = covid_goiania_por_bairro.T
bairros_e_num_casos.to_csv('covid_goiania_por_bairro.csv', index=False)
covid_goiania_por_bairro

,data,bairro,num_casos
0,2020-05-02,Setor Bueno,48
1,2020-05-02,Setor Oeste,39
2,2020-05-02,Jd. Goiás,18
3,2020-05-02,Setor Marista,14
4,2020-05-02,Parque Amazônia,14
...,...,...,...
8549,2020-09-18,Res. Granville,132
8550,2020-09-18,Vila Mutirão I,132
8551,2020-09-18,Conjunto Habitacional Madre Germana 2,128
8552,2020-09-18,Vila Jardim Pompéia,128


In [ ]:
#quantidade de bairros
# 2 = "outros " e "outros bairros"

print(len(np.unique(covid_goiania_por_bairro['bairro'])) - 2)


114


In [ ]:
covid_goiania_por_bairro.to_csv('covid_goiania_por_bairro.csv', index=False)


## Web scraping (CEPs) e obtenção de coordenadas


In [ ]:
# extrair coordenadas

In [ ]:
# bairros_e_localizacao = pd.read_csv('bairros_e_localizacao.csv', sep=',')
# bairros_e_localizacao.to_csv('bairros_e_localizacao.csv', index=False)
bairros_e_localizacao

,bairro,cep,localizacao
0,Alphaville Flamboyant,74884-534,"[-16.7070932, -49.2122502]"
1,Alto da Glória,74815-610,"[-16.7206511, -49.2250865]"
2,Capuava,74450-410,"[-16.6645242, -49.316575]"
3,Cidade Jardim,74425-250,"[-16.6761304, -49.3121233]"
4,Conjunto Primavera,74477-207,"[-16.5878046, -49.37404129999999]"
...,...,...,...
106,Vila São José - Extensão,70000-000,"[-16.6551261,-49.3078991]"
107,Setor Rio Formoso,74370-020,"[-16.7211142,-49.3337986]"
108,Residencial Jardim Florença,74351-008,"[-16.725215,-49.3151065]"
109,Setor Perim,74580-240,"[-16.6448258,-49.2994181]"


## Merge covid_goiania_por_bairro e bairros_e_localizacao

In [ ]:
covid_goiania_por_bairro = pd.merge(covid_goiania_por_bairro, bairros_e_localizacao, on='bairro')
covid_goiania_por_bairro

,data,bairro,num_casos,cep,localizacao
0,2020-05-02,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]"
1,2020-04-29,Setor Bueno,45,74223-010,"[-16.6956937, -49.2640252]"
2,2020-04-30,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]"
3,2020-05-03,Setor Bueno,52,74223-010,"[-16.6956937, -49.2640252]"
4,2020-05-11,Setor Bueno,73,74223-010,"[-16.6956937, -49.2640252]"
...,...,...,...,...,...
8251,2020-09-17,Conjunto Habitacional Madre Germana 2,128,74354-830,"[-16.8154446,-49.3747038]"
8252,2020-09-18,Conjunto Habitacional Madre Germana 2,128,74354-830,"[-16.8154446,-49.3747038]"
8253,2020-09-16,Setor Rio Formoso,125,74370-020,"[-16.7211142,-49.3337986]"
8254,2020-09-17,Setor Rio Formoso,125,74370-020,"[-16.7211142,-49.3337986]"


## Acrescentar regiao em covid_goiania_por_bairro

In [ ]:
#web scraping de http://www.sgc.goias.gov.br/upload/arquivos/2017-02/bairros-por-regiao-2017.pdf
# ou do wikipedia (code disponivel na v1)

In [ ]:
bairros_por_regiao = pd.read_csv('bairros_por_regiao.csv', encoding = 'utf-8', sep=',')


In [ ]:
bairros_por_regiao

,bairro,regiao
0,Alice Barbosa,Norte
1,Antônio Barbosa,Norte
2,Antônio Carlos Pires,Norte
3,Asa Branca,Norte
4,Atalaia,Norte
...,...,...
488,Vila São José - Extensão,Centro Oeste
489,Setor Rio Formoso,Sudoeste
490,Residencial Jardim Florença,Sudoeste
491,Setor Perim,Norte


In [ ]:
def normaliza_nomes_bairros(bairro):

    if 'Res.' in bairro:
        bairro = bairro.replace('Res.', 'Residencial')
        # bairro = bairro.strip(' ')

        return bairro
    
    if 'Jd.' in bairro:
        bairro = bairro.replace('Jd.', 'Jardim')
        # bairro = bairro.strip(' ')
        return bairro
    else:
        return bairro 

covid_goiania_por_bairro['bairro'] = covid_goiania_por_bairro['bairro'].apply(lambda x: normaliza_nomes_bairros(x))
covid_goiania_por_bairro['bairro'] = covid_goiania_por_bairro['bairro'].apply(lambda x: x.strip(' '))
df_completo_com_regiao = pd.merge(covid_goiania_por_bairro,  bairros_por_regiao, on='bairro')
df_completo_com_regiao = df_completo_com_regiao.drop_duplicates()
df_completo_com_regiao


,data,bairro,num_casos,cep,localizacao,regiao
0,2020-05-02,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]",Sul
1,2020-04-29,Setor Bueno,45,74223-010,"[-16.6956937, -49.2640252]",Sul
2,2020-04-30,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]",Sul
3,2020-05-03,Setor Bueno,52,74223-010,"[-16.6956937, -49.2640252]",Sul
4,2020-05-11,Setor Bueno,73,74223-010,"[-16.6956937, -49.2640252]",Sul
...,...,...,...,...,...,...
11399,2020-09-17,Conjunto Habitacional Madre Germana 2,128,74354-830,"[-16.8154446,-49.3747038]",Noroeste
11400,2020-09-18,Conjunto Habitacional Madre Germana 2,128,74354-830,"[-16.8154446,-49.3747038]",Noroeste
11401,2020-09-16,Setor Rio Formoso,125,74370-020,"[-16.7211142,-49.3337986]",Sudoeste
11402,2020-09-17,Setor Rio Formoso,125,74370-020,"[-16.7211142,-49.3337986]",Sudoeste


In [ ]:
bairros_c_regiao = np.unique(df_completo_com_regiao['bairro'])
bairros_covid = np.unique(covid_goiania_por_bairro['bairro'])

faltantes = list(set(bairros_covid).difference(bairros_c_regiao))
faltantes

[]

In [ ]:
len(np.unique(df_completo_com_regiao['bairro']))

109

In [ ]:
df_completo_com_regiao.to_csv('covid_goiania_por_bairro.csv', index=False)
covid_goiania_por_bairro = df_completo_com_regiao

## Bar race - região

In [ ]:
 df_bar_race = covid_goiania_por_bairro
 regioes = np.unique(df_bar_race['regiao'])
 lista_regioes = list(regioes)
 lista_regioes

['Centro Oeste', 'Leste', 'Noroeste', 'Norte', 'Oeste', 'Sudoeste', 'Sul']

In [ ]:
indices = []
for i, linha in df_bar_race.iterrows():
    if linha[1] in lista_regioes:
        indices.append(i)

df_bar_race = df_bar_race.drop(indices)
df_bar_race

,data,bairro,num_casos,cep,localizacao,regiao
0,2020-05-02,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]",Sul
1,2020-04-29,Setor Bueno,45,74223-010,"[-16.6956937, -49.2640252]",Sul
2,2020-04-30,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]",Sul
3,2020-05-03,Setor Bueno,52,74223-010,"[-16.6956937, -49.2640252]",Sul
4,2020-05-11,Setor Bueno,73,74223-010,"[-16.6956937, -49.2640252]",Sul
...,...,...,...,...,...,...
11399,2020-09-17,Conjunto Habitacional Madre Germana 2,128,74354-830,"[-16.8154446,-49.3747038]",Noroeste
11400,2020-09-18,Conjunto Habitacional Madre Germana 2,128,74354-830,"[-16.8154446,-49.3747038]",Noroeste
11401,2020-09-16,Setor Rio Formoso,125,74370-020,"[-16.7211142,-49.3337986]",Sudoeste
11402,2020-09-17,Setor Rio Formoso,125,74370-020,"[-16.7211142,-49.3337986]",Sudoeste


In [ ]:
df_bar_race.groupby(['regiao', 'data']).sum()['num_casos']

regiao        data      
Centro Oeste  2020-04-29      44
              2020-04-30      45
              2020-05-01      45
              2020-05-02      50
              2020-05-03      47
                            ... 
Sul           2020-09-14    7596
              2020-09-15    7596
              2020-09-16    8152
              2020-09-17    8152
              2020-09-18    8152
Name: num_casos, Length: 914, dtype: int64

In [ ]:
df_bar_race = pd.DataFrame(df_bar_race.groupby(['regiao', 'data']).sum()['num_casos']).unstack().T.droplevel(level=0)

In [ ]:
df_bar_race.columns = ['Centro', 'Leste', 'Noroeste', 'Norte', 'Oeste', 'Sudoeste', 'Sul']
bcr.bar_chart_race(df_bar_race, filename='bar_race_regiao.mp4')


## Bar race - bairro

In [ ]:
df_bar_race = pd.read_csv('covid_goiania_por_bairro.csv', encoding = 'utf-8', sep=',')


In [ ]:
 bairros = np.unique(df_bar_race['bairro'])
 lista_bairros = list(bairros)
lista_bairros


['Alphaville Flamboyant',
 'Alto da Glória',
 'Capuava',
 'Cidade Jardim',
 'Conjunto Habitacional Madre Germana 2',
 'Conjunto Primavera',
 'Conjunto Vera Cruz',
 'Faicalville',
 'Floresta',
 'Goiá',
 'Goiânia 2',
 'Jardim América',
 'Jardim Atlântico',
 'Jardim Balneário Meia Ponte',
 'Jardim Bela Vista',
 'Jardim Curitiba',
 'Jardim Europa',
 'Jardim Goiás',
 'Jardim Guanabara',
 'Jardim Guanabara II',
 'Jardim Guanabara III',
 'Jardim Itaipu',
 'Jardim Liberdade',
 'Jardim Mariliza',
 'Jardim Nova Esperanca',
 'Jardim Novo Mundo',
 'Jardim Petrópolis',
 'Jardim Planalto',
 'Jardim Presidente',
 'Jardim Vila Boa',
 'Jardim das Esmeraldas',
 'Nova Suiça',
 'Parque Amazônia',
 'Parque Anhanguera',
 'Parque Atheneu',
 'Parque Industrial de Goiânia',
 'Parque Oeste Industrial',
 'Parque Santa Rita',
 'Parque das Amendoeiras',
 'Parque das Laranjeiras',
 'Portal do Sol II',
 'Residencial Aldeia do Vale',
 'Residencial Brisas da Mata',
 'Residencial Eldorado',
 'Residencial Goiânia Viva',

In [ ]:
#lista de bairros que não irão aparecer no bar race 
#bairros removidos da lista:

lista_bairros = ['Alphaville Flamboyant',
 'Alto da Glória',
 'Capuava',
 'Cidade Jardim',
 'Conjunto Habitacional Madre Germana 2',
 'Conjunto Primavera',
 'Conjunto Vera Cruz',
 'Faicalville',
 'Floresta',
 'Goiá',
 'Jardim América',
 'Jardim Atlântico',
 'Jardim Balneário Meia Ponte',
 'Jardim Bela Vista',
 'Jardim Curitiba',
 'Jardim Europa',
 'Jardim Guanabara II',
 'Jardim Guanabara III',
 'Jardim Itaipu',
 'Jardim Liberdade',
 'Jardim Mariliza',
 'Jardim Nova Esperanca',
 'Jardim Novo Mundo',
 'Jardim Petrópolis',
 'Jardim Planalto',
 'Jardim Presidente',
 'Jardim São José',
 'Jardim Valência',
 'Jardim Vila Boa',
 'Jardim das Esmeraldas',
 'Nova Suiça',
 'Parque Anhanguera',
 'Parque Atheneu',
 'Parque Industrial João Braz',
 'Parque Industrial de Goiânia',
 'Parque Oeste Industrial',
 'Parque Santa Rita',
 'Parque das Amendoeiras',
 'Parque das Laranjeiras',
 'Portal do Sol II',
 'Residencial Aldeia do Vale',
 'Residencial Brisas da Mata',
 'Residencial Eldorado',
 'Residencial Goiânia Viva',
 'Residencial Granville',
 'Residencial Itaipú',
 'Residencial Jardim Florença',
 'Residencial Orlando de Morais',
 'Residencial Petropolis',
 'Residencial Recanto das Garças',
 'Residencial Vale dos Sonhos I',
 'Rodoviário',
 'Santa Genoveva',
 'Santo Hilário',
 'Setor Aeroporto',
 'Setor Bela Vista',
 'Setor Candida de Morais',
 'Setor Centro Oeste',
 'Setor Criméia Leste',
 'Setor Criméia Oeste',
 'Setor Estrela Dalva',
 'Setor Garavelo',
 'Setor Jaó',
 'Setor Marechal Rondon',
 'Setor Marista',
 'Setor Morada do Sol',
 'Setor Novo Horizonte',
 'Setor Novo Planalto',
 'Setor Parque Tremendão',
 'Setor Perim',
 'Setor Recanto das Minas Gerais',
 'Setor Rio Formoso',
 'Setor Santos Dumont',
 'Setor Sudoeste',
 'Setor Sul',
 'Setor São Jose',
 'Setor União',
 'Setor Urias Magalhaes',
 'Setor dos Funcionários',
 'Solange Parque I',
 'São Carlos',
 'São Francisco',
 'Vila Alpes',
 'Vila Canaa',
 'Vila Concórdia',
 'Vila Finsocial',
 'Vila Jardim São Judas Tadeu',
 'Vila João Vaz',
 'Vila Maria José',
 'Vila Mutirão I',
 'Vila Pedroso',
 'Vila Redenção',
 'Vila Regina',
 'Vila Rosa',
 'Vila Santa Helena',
 'Vila São José - Extensão',
 'da Vitória']

In [ ]:
len(lista_bairros)

97

In [ ]:
#remove os bairros
indices = []
for i, linha in df_bar_race.iterrows():
    if linha['bairro'] in lista_bairros:
        indices.append(i)



In [ ]:
df_bar_race = df_bar_race.drop(indices)
df_bar_race

,data,bairro,num_casos,cep,localizacao,regiao
0,2020-05-02,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]",Sul
1,2020-04-29,Setor Bueno,45,74223-010,"[-16.6956937, -49.2640252]",Sul
2,2020-04-30,Setor Bueno,48,74223-010,"[-16.6956937, -49.2640252]",Sul
3,2020-05-03,Setor Bueno,52,74223-010,"[-16.6956937, -49.2640252]",Sul
4,2020-05-11,Setor Bueno,73,74223-010,"[-16.6956937, -49.2640252]",Sul
...,...,...,...,...,...,...
7975,2020-09-14,Vila Itatiaia,119,74690-505,"[-16.5957572, -49.258295]",Norte
7976,2020-09-15,Vila Itatiaia,119,74690-505,"[-16.5957572, -49.258295]",Norte
7977,2020-09-16,Vila Itatiaia,138,74690-505,"[-16.5957572, -49.258295]",Norte
7978,2020-09-17,Vila Itatiaia,138,74690-505,"[-16.5957572, -49.258295]",Norte


In [ ]:
df_bar_race.groupby(['bairro', 'data']).sum()['num_casos']

bairro               data      
Goiânia 2            2020-05-11      5
                     2020-05-12      5
                     2020-05-14      6
                     2020-05-15      7
                     2020-05-16      7
                                  ... 
Vila Jardim Pompéia  2020-09-14    118
                     2020-09-15    118
                     2020-09-16    128
                     2020-09-17    128
                     2020-09-18    128
Name: num_casos, Length: 1888, dtype: int64

In [ ]:
df_bar_race = pd.DataFrame(df_bar_race.groupby(['bairro', 'data']).sum()['num_casos']).unstack().T.droplevel(level=0)
bcr.bar_chart_race(df_bar_race, filename='bar_race_bairro.mp4')


## Dataframe para plotagem do mapa

In [ ]:
covid_goiania_por_bairro.columns = ['Date', 'Zone', 'Count', 'CEP', 'Location', 'Region']
covid_goiania_por_bairro['Category'] = 'Confirmed'
covid_goiania_por_bairro["Count"] = pd.to_numeric(covid_goiania_por_bairro["Count"])

covid_goiania_por_bairro['Location'] = covid_goiania_por_bairro['Location'].apply(lambda x: x[1:])
covid_goiania_por_bairro['Location'] = covid_goiania_por_bairro['Location'].apply(lambda x: x[:-1])
covid_goiania_por_bairro['Location'] = covid_goiania_por_bairro['Location'].apply(lambda x: x.replace(" ", ""))

# df_completo.to_csv('/content/drive/My Drive/Colab Notebooks/covid/covid-19-goiania-por-bairro-final.csv', index=False)
covid_goiania_por_bairro.to_csv('covid-19-goiania-por-bairro-final.csv', index=False)
covid_goiania_por_bairro

,Date,Zone,Count,CEP,Location,Region,Category
0,2020-05-02,Setor Bueno,48,74223-010,"-16.6956937,-49.2640252",Sul,Confirmed
1,2020-04-29,Setor Bueno,45,74223-010,"-16.6956937,-49.2640252",Sul,Confirmed
2,2020-04-30,Setor Bueno,48,74223-010,"-16.6956937,-49.2640252",Sul,Confirmed
3,2020-05-03,Setor Bueno,52,74223-010,"-16.6956937,-49.2640252",Sul,Confirmed
4,2020-05-11,Setor Bueno,73,74223-010,"-16.6956937,-49.2640252",Sul,Confirmed
...,...,...,...,...,...,...,...
11399,2020-09-17,Conjunto Habitacional Madre Germana 2,128,74354-830,"-16.8154446,-49.3747038",Noroeste,Confirmed
11400,2020-09-18,Conjunto Habitacional Madre Germana 2,128,74354-830,"-16.8154446,-49.3747038",Noroeste,Confirmed
11401,2020-09-16,Setor Rio Formoso,125,74370-020,"-16.7211142,-49.3337986",Sudoeste,Confirmed
11402,2020-09-17,Setor Rio Formoso,125,74370-020,"-16.7211142,-49.3337986",Sudoeste,Confirmed


## Gráfico de barras



In [ ]:
#fonte dos dados: https://brasil.io/dataset/covid19/caso/?search=Goi%C3%A2nia&date=&state=GO&city=&place_type=&is_last=&city_ibge_code=5208707&order_for_place=



bar_graph_dados = pd.read_csv('covid19-335f39aca2a64a1a99ef2e3e5eedbbc0.csv', sep=',')

In [ ]:
bar_graph_dados.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-09-17,GO,Goiânia,city,45662,1168,True,1516113,5208707,3011.78078,0.0256
1,2020-09-16,GO,Goiânia,city,44324,1135,False,1516113,5208707,2923.52879,0.0256
2,2020-09-15,GO,Goiânia,city,42978,1116,False,1516113,5208707,2834.74912,0.0260
3,2020-09-14,GO,Goiânia,city,41698,1098,False,1516113,5208707,2750.32270,0.0263
4,2020-09-13,GO,Goiânia,city,39797,1074,False,1516113,5208707,2624.93627,0.0270


In [ ]:
bar_graph_dados.columns

Index(['date', 'state', 'city', 'place_type', 'confirmed', 'deaths', 'is_last',
       'estimated_population_2019', 'city_ibge_code',
       'confirmed_per_100k_inhabitants', 'death_rate'],
      dtype='object')

In [ ]:
bar_graph_dados.drop(columns=['is_last', 'estimated_population_2019', 'city_ibge_code', 'state',
                                'confirmed_per_100k_inhabitants', 'death_rate','place_type'
                            ], inplace=True)
                     

bar_graph_dados.head()

,date,city,confirmed,deaths
0,2020-09-17,Goiânia,45662,1168
1,2020-09-16,Goiânia,44324,1135
2,2020-09-15,Goiânia,42978,1116
3,2020-09-14,Goiânia,41698,1098
4,2020-09-13,Goiânia,39797,1074


In [ ]:
bar_graph_dados.rename(columns={'date': 'Data', 'city': 'Cidade', 'confirmed': 'Casos confirmados', 'deaths': 'Óbitos'}, inplace=True)
bar_graph_dados.head()

,Data,Cidade,Casos confirmados,Óbitos
0,2020-09-17,Goiânia,45662,1168
1,2020-09-16,Goiânia,44324,1135
2,2020-09-15,Goiânia,42978,1116
3,2020-09-14,Goiânia,41698,1098
4,2020-09-13,Goiânia,39797,1074


In [ ]:
fig = px.bar(bar_graph_dados, x='Data', y='Casos confirmados')
fig.update_traces()

In [ ]:
fig = px.bar(bar_graph_dados, x='Data', y='Óbitos')
fig.update_traces()

In [ ]:
# bar_graph_dados.to_csv('/content/drive/My Drive/Colab Notebooks/covid/brasil_io-goiania.csv', index=False)
bar_graph_dados.to_csv('brasil_io-goiania.csv', index=False)

# Etapas a partir dos dados da Secretária de Saúde do Estado de Goiás

*   Gráfico de barras - casos ativos por data de notificação
*   Gráfico de barras - casos confirmados por data de notificação
*   Gráfico de barras - óbitos por data de notificação
*   Média móvel em todos os gráficos




## Dataset - casos

```
# casos_goiania_cidade
```



In [2]:
casos_goiania_cidade = pd.read_csv('http://datasets.saude.go.gov.br/coronavirus/casos_confirmados.csv', sep=';')
casos_goiania_cidade

,data_notificacao,data_inicio_sintomas,ano_epi,semana_epi,sexo,raca_cor,faixa_etaria,diabetes,doenca_cardiovascular,doenca_respiratoria,imunossupressao,codigo_ibge,municipio,regiao_saude,recuperado
0,20200706,20200623,2020,26,FEMININO,PARDA,60 a 69 anos,NAO,NAO,NAO,NAO,520753,FAINA,RIO VERMELHO,SIM
1,20200619,20200613,2020,24,FEMININO,PARDA,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,522140,TRINDADE,CENTRAL,SIM
2,20200709,20200706,2020,28,FEMININO,BRANCA,>= 80 anos,SIM,SIM,NAO,NAO,520549,CIDADE OCIDENTAL,ENTORNO SUL,NAO
3,20200628,20200620,2020,25,FEMININO,AMARELA,60 a 69 anos,NAO,NAO,NAO,NAO,520140,APARECIDA DE GOIANIA,CENTRO SUL,SIM
4,20200715,20200712,2020,29,FEMININO,PARDA,50 a 59 anos,NAO,NAO,NAO,NAO,520140,APARECIDA DE GOIANIA,CENTRO SUL,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183722,20200730,20200730,2020,31,MASCULINO,PARDA,30 a 39 anos,NAO,NAO,NAO,NAO,520140,APARECIDA DE GOIANIA,CENTRO SUL,SIM
183723,20200714,20200705,2020,28,MASCULINO,PARDA,< 10 anos,NAO,NAO,NAO,NAO,520549,CIDADE OCIDENTAL,ENTORNO SUL,SIM
183724,20200727,20200711,2020,28,MASCULINO,BRANCA,20 a 29 anos,NAO,NAO,NAO,NAO,521930,SANTA HELENA DE GOIAS,SUDOESTE I,SIM
183725,20200909,20200907,2020,37,MASCULINO,IGNORADO,< 10 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,NAO


## Óbitos por dia

*   Gráfico de barras com média móvel




```
dataframe: obitos_por_dia
gera: obitos_por_dia_goiania.csv
```


In [106]:
obitos_por_dia = pd.read_csv('http://datasets.saude.go.gov.br/coronavirus/obitos_confirmados.csv', sep=';')
obitos_por_dia_remove = obitos_por_dia.loc[((obitos_por_dia['municipio'] != 'GOIANIA'))]

obitos_por_dia = obitos_por_dia.drop(obitos_por_dia_remove.index)
obitos_por_dia

,data_notificacao,data_inicio_sintomas,ano_epi,semana_epi,sexo,raca_cor,faixa_etaria,diabetes,doenca_cardiovascular,doenca_respiratoria,imunossupressao,codigo_ibge,municipio,regiao_saude,data_obito
8,20200901,20200801,2020,31,FEMININO,IGNORADO,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200905
28,20200629,20200618,2020,25,FEMININO,BRANCA,>= 80 anos,NAO,SIM,SIM,NAO,520870,GOIANIA,CENTRAL,20200702
29,20200806,20200802,2020,32,FEMININO,IGNORADO,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200809
30,20200715,20200627,2020,26,FEMININO,PARDA,50 a 59 anos,NAO,SIM,SIM,NAO,520870,GOIANIA,CENTRAL,20200709
33,20200803,20200731,2020,31,FEMININO,IGNORADO,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131,20200720,20200717,2020,29,FEMININO,BRANCA,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200726
4138,20200810,20200727,2020,31,MASCULINO,IGNORADO,50 a 59 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200825
4146,20200701,20200622,2020,26,MASCULINO,BRANCA,50 a 59 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,20200712
4147,20200618,20200613,2020,24,MASCULINO,IGNORADO,50 a 59 anos,IGNORADO,SIM,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200626


In [107]:
obitos_por_dia = obitos_por_dia['data_notificacao'].value_counts() 
obitos_por_dia = obitos_por_dia.to_frame(name='num_casos')
obitos_por_dia.reset_index(level=0, inplace=True)
obitos_por_dia.rename(columns={'index': 'data_notificacao'}, inplace = True)
obitos_por_dia

,data_notificacao,num_casos
0,20200731,33
1,20200805,25
2,20200803,24
3,20200901,22
4,20200701,21
...,...,...
142,20200514,1
143,20200516,1
144,20200609,1
145,20200913,1


In [108]:
obitos_por_dia['data_notificacao'] = obitos_por_dia['data_notificacao'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d').date())

obitos_por_dia


,data_notificacao,num_casos
0,2020-07-31,33
1,2020-08-05,25
2,2020-08-03,24
3,2020-09-01,22
4,2020-07-01,21
...,...,...
142,2020-05-14,1
143,2020-05-16,1
144,2020-06-09,1
145,2020-09-13,1


In [109]:
obitos_por_dia['data_notificacao'] = pd.to_datetime(obitos_por_dia['data_notificacao'])

# Ordena pela data e salva em um novo DataFrame
obitos_por_dia = obitos_por_dia.sort_values(by='data_notificacao')
obitos_por_dia

,data_notificacao,num_casos
146,2020-04-01,1
128,2020-04-03,1
119,2020-04-04,2
116,2020-04-05,2
126,2020-04-10,1
...,...,...
145,2020-09-13,1
100,2020-09-14,4
113,2020-09-15,3
124,2020-09-16,2


In [110]:
obitos_por_dia.to_csv('obitos_por_dia_goiania.csv', index=False)
obitos_por_dia

,data_notificacao,num_casos
146,2020-04-01,1
128,2020-04-03,1
119,2020-04-04,2
116,2020-04-05,2
126,2020-04-10,1
...,...,...
145,2020-09-13,1
100,2020-09-14,4
113,2020-09-15,3
124,2020-09-16,2


### Média móvel - casos confirmados por dia

In [127]:
# casos_ativos_goiania = pd.read_csv('casos_ativos_goiania.csv', sep=',')

In [128]:
df = obitos_por_dia
for i in range(0,df.shape[0]-6):
    df.loc[df.index[i+6],'MM_casos'] = np.round(((df.iloc[i,1]+
                                                  df.iloc[i+1,1]+
                                                  df.iloc[i+2,1]+
                                                  df.iloc[i+3,1]+
                                                  df.iloc[i+4,1]+
                                                  df.iloc[i+5,1]+
                                                  df.iloc[i+6,1])/7),1)           

df.head(8)

,data_notificacao,num_casos,MM_casos
146,2020-04-01,1,NaN
128,2020-04-03,1,NaN
119,2020-04-04,2,NaN
116,2020-04-05,2,NaN
126,2020-04-10,1,NaN
127,2020-04-14,1,NaN
129,2020-04-15,1,1.3
130,2020-04-16,1,1.3


In [129]:
obitos_por_dia.to_csv('obitos_por_dia_goiania.csv', index=False)

In [130]:
fig = px.bar(obitos_por_dia, x='data_notificacao', y='num_casos',  title="Óbitos por dia por data de notificação", labels={'num_casos':'N° de óbitos', 
                                                                         'data_notificacao': 'Data de notificação',})

fig.add_trace(
    go.Scatter(
        x=obitos_por_dia['data_notificacao'],
        y=obitos_por_dia['MM_casos'],
        name='Média Móvel<br>(7 dias)',        
        hovertemplate= "Média Móvel: %{x}<br>Data: %{y}<extra></extra>",

    ))

fig.update_traces()

In [131]:
import plotly.graph_objects as go
fig.show()
fig.write_image("obitos_por_dia.svg")
fig.write_image("obitos_por_dia.png")

In [132]:
from google.colab import files
files.download('obitos_por_dia.svg')
files.download('obitos_por_dia.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Casos ativos

*   Gráfico de barras com média móvel




```
dataframe = classificacdos como "nao recuperado" - obtos_por_dia
gera: casos_ativos_goiania.csv
```



In [86]:
nao_recuperados = casos_goiania_cidade
nao_recuperados_remove = nao_recuperados.loc[((nao_recuperados['recuperado'] != 'NAO') | (nao_recuperados['municipio'] != 'GOIANIA'))]
# df_remove = df.loc[(df['recuperado'] != 'NAO')]

nao_recuperados = nao_recuperados.drop(nao_recuperados_remove.index)
nao_recuperados.reset_index(level=0, inplace=True)
nao_recuperados = nao_recuperados.drop(columns=['index'])
nao_recuperados

,data_notificacao,data_inicio_sintomas,ano_epi,semana_epi,sexo,raca_cor,faixa_etaria,diabetes,doenca_cardiovascular,doenca_respiratoria,imunossupressao,codigo_ibge,municipio,regiao_saude,recuperado
0,20200805,20200728,2020,31,FEMININO,IGNORADO,>= 80 anos,NAO,NAO,SIM,NAO,520870,GOIANIA,CENTRAL,NAO
1,20200804,20200729,2020,31,FEMININO,IGNORADO,>= 80 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,NAO
2,20200811,20200806,2020,32,FEMININO,PARDA,>= 80 anos,NAO,SIM,NAO,NAO,520870,GOIANIA,CENTRAL,NAO
3,20200731,20200728,2020,31,MASCULINO,PARDA,>= 80 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,NAO
4,20200704,20200628,2020,27,MASCULINO,AMARELA,70 a 79 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,NAO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2797,20200829,20200825,2020,35,FEMININO,IGNORADO,>= 80 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,NAO
2798,20200917,20200911,2020,37,FEMININO,IGNORADO,60 a 69 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,NAO
2799,20200909,20200907,2020,37,MASCULINO,BRANCA,20 a 29 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,NAO
2800,20200911,20200906,2020,37,MASCULINO,AMARELA,20 a 29 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,NAO


In [87]:
nao_recuperados = nao_recuperados['data_notificacao'].value_counts() 
nao_recuperados = nao_recuperados.to_frame(name='num_casos')
nao_recuperados.reset_index(level=0, inplace=True)
nao_recuperados.rename(columns={'index': 'data_notificacao'}, inplace = True)
nao_recuperados

,data_notificacao,num_casos
0,20200915,242
1,20200914,190
2,20200916,122
3,20200918,112
4,20200911,108
...,...,...
146,20200420,1
147,20200416,1
148,20200414,1
149,20200410,1


In [88]:
nao_recuperados['data_notificacao'] = nao_recuperados['data_notificacao'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d').date())

nao_recuperados


,data_notificacao,num_casos
0,2020-09-15,242
1,2020-09-14,190
2,2020-09-16,122
3,2020-09-18,112
4,2020-09-11,108
...,...,...
146,2020-04-20,1
147,2020-04-16,1
148,2020-04-14,1
149,2020-04-10,1


In [73]:
obitos_por_dia = pd.read_csv('http://datasets.saude.go.gov.br/coronavirus/obitos_confirmados.csv', sep=';')
obitos_por_dia_remove = obitos_por_dia.loc[((obitos_por_dia['municipio'] != 'GOIANIA'))]

obitos_por_dia = obitos_por_dia.drop(obitos_por_dia_remove.index)
obitos_por_dia.reset_index(level=0, inplace=True)
obitos_por_dia = obitos_por_dia.drop(columns=['index'])

obitos_por_dia

,data_notificacao,data_inicio_sintomas,ano_epi,semana_epi,sexo,raca_cor,faixa_etaria,diabetes,doenca_cardiovascular,doenca_respiratoria,imunossupressao,codigo_ibge,municipio,regiao_saude,data_obito
0,20200901,20200801,2020,31,FEMININO,IGNORADO,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200905
1,20200629,20200618,2020,25,FEMININO,BRANCA,>= 80 anos,NAO,SIM,SIM,NAO,520870,GOIANIA,CENTRAL,20200702
2,20200806,20200802,2020,32,FEMININO,IGNORADO,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200809
3,20200715,20200627,2020,26,FEMININO,PARDA,50 a 59 anos,NAO,SIM,SIM,NAO,520870,GOIANIA,CENTRAL,20200709
4,20200803,20200731,2020,31,FEMININO,IGNORADO,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,20200720,20200717,2020,29,FEMININO,BRANCA,60 a 69 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200726
1198,20200810,20200727,2020,31,MASCULINO,IGNORADO,50 a 59 anos,IGNORADO,IGNORADO,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200825
1199,20200701,20200622,2020,26,MASCULINO,BRANCA,50 a 59 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,20200712
1200,20200618,20200613,2020,24,MASCULINO,IGNORADO,50 a 59 anos,IGNORADO,SIM,IGNORADO,IGNORADO,520870,GOIANIA,CENTRAL,20200626


In [74]:
obitos_por_dia = obitos_por_dia['data_notificacao'].value_counts() 
obitos_por_dia = obitos_por_dia.to_frame(name='num_casos')
obitos_por_dia.reset_index(level=0, inplace=True)
obitos_por_dia.rename(columns={'index': 'data_notificacao'}, inplace = True)
obitos_por_dia

,data_notificacao,num_casos
0,20200731,33
1,20200805,25
2,20200803,24
3,20200901,22
4,20200701,21
...,...,...
142,20200514,1
143,20200516,1
144,20200609,1
145,20200913,1


In [75]:
obitos_por_dia['data_notificacao'] = obitos_por_dia['data_notificacao'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d').date())

obitos_por_dia


,data_notificacao,num_casos
0,2020-07-31,33
1,2020-08-05,25
2,2020-08-03,24
3,2020-09-01,22
4,2020-07-01,21
...,...,...
142,2020-05-14,1
143,2020-05-16,1
144,2020-06-09,1
145,2020-09-13,1


In [89]:
faltantes = []
casos_ativos = {}

k = 0
for i, linha_ob in obitos_por_dia.iterrows():
    # print(linha_ob['data_notificacao'])
    for j, linha_nao_rec in nao_recuperados.iterrows():
        if linha_ob['data_notificacao'] == linha_nao_rec['data_notificacao']:
            num= linha_nao_rec['num_casos'] - linha_ob['num_casos']
            casos_ativos[k] = {'data': linha_nao_rec['data_notificacao'], 'num_casos': num}
            faltantes.append(j)
            k+=1
            break


In [90]:
casos_ativos=pd.DataFrame(data=casos_ativos)
casos_ativos= casos_ativos.T
casos_ativos.reset_index(level=0, inplace=True)
casos_ativos.drop(columns=['index'], inplace = True)
casos_ativos.head

<bound method NDFrame.head of            data num_casos
0    2020-07-31         5
1    2020-08-05         3
2    2020-08-03         9
3    2020-09-01        13
4    2020-07-01         2
..          ...       ...
142  2020-05-14         0
143  2020-05-16         1
144  2020-06-09         0
145  2020-09-13        51
146  2020-04-01         0

[147 rows x 2 columns]>

In [92]:
nao_recuperados =nao_recuperados.drop(faltantes)
nao_recuperados.reset_index(level=0, inplace=True)
nao_recuperados.drop(columns=['index'], inplace = True)
nao_recuperados

,data_notificacao,num_casos
0,2020-09-17,92
1,2020-09-07,8
2,2020-09-19,1
3,2020-04-24,1


In [93]:
casos_ativos = casos_ativos.to_dict('dict')
nao_recuperados = nao_recuperados.to_dict('dict')

In [94]:
ativos = {}

j= 0
for i in range(len(casos_ativos['num_casos'])):
    ativos[j] = {'data_notificacao': casos_ativos['data'][i], 'num_casos': casos_ativos['num_casos'][i]}
    j+=1

for i in range(len(nao_recuperados['num_casos'])):
    ativos[j] = {'data_notificacao': nao_recuperados['data_notificacao'][i], 'num_casos': nao_recuperados['num_casos'][i]}
    j+=1



In [95]:
casos_ativos_goiania = pd.DataFrame(ativos)
casos_ativos_goiania = casos_ativos_goiania.T
casos_ativos_goiania

,data_notificacao,num_casos
0,2020-07-31,5
1,2020-08-05,3
2,2020-08-03,9
3,2020-09-01,13
4,2020-07-01,2
...,...,...
146,2020-04-01,0
147,2020-09-17,92
148,2020-09-07,8
149,2020-09-19,1


In [96]:
sum(casos_ativos_goiania['num_casos'])

1600

In [97]:
casos_ativos_goiania['data_notificacao'] = pd.to_datetime(casos_ativos_goiania['data_notificacao'])

# Ordena pela data e salva em um novo DataFrame
casos_ativos_goiania = casos_ativos_goiania.sort_values(by='data_notificacao')
casos_ativos_goiania

,data_notificacao,num_casos
146,2020-04-01,0
128,2020-04-03,0
119,2020-04-04,0
116,2020-04-05,0
126,2020-04-10,0
...,...,...
113,2020-09-15,239
124,2020-09-16,120
147,2020-09-17,92
135,2020-09-18,111


In [98]:
casos_ativos_goiania.to_csv('casos_ativos_goiania.csv', index=False)
casos_ativos_goiania


,data_notificacao,num_casos
146,2020-04-01,0
128,2020-04-03,0
119,2020-04-04,0
116,2020-04-05,0
126,2020-04-10,0
...,...,...
113,2020-09-15,239
124,2020-09-16,120
147,2020-09-17,92
135,2020-09-18,111


### Média móvel - ativos

In [133]:
casos_ativos_goiania = pd.read_csv('casos_ativos_goiania.csv', sep=',')

In [134]:
df = casos_ativos_goiania
for i in range(0,df.shape[0]-6):
    df.loc[df.index[i+6],'MM_casos'] = np.round(((df.iloc[i,1]+
                                                  df.iloc[i+1,1]+
                                                  df.iloc[i+2,1]+
                                                  df.iloc[i+3,1]+
                                                  df.iloc[i+4,1]+
                                                  df.iloc[i+5,1]+
                                                  df.iloc[i+6,1])/7),1)           

df.head

<bound method NDFrame.head of     data_notificacao  num_casos  MM_casos
0         2020-04-01          0       NaN
1         2020-04-03          0       NaN
2         2020-04-04          0       NaN
3         2020-04-05          0       NaN
4         2020-04-10          0       NaN
..               ...        ...       ...
146       2020-09-15        239     118.6
147       2020-09-16        120     121.3
148       2020-09-17         92     121.1
149       2020-09-18        111     122.1
150       2020-09-19          1     114.3

[151 rows x 3 columns]>

In [135]:
casos_ativos_goiania.to_csv('casos_ativos_goiania.csv', index=False)

In [136]:
fig = px.bar(casos_ativos_goiania, x='data_notificacao', y='num_casos',  title="Casos ativos por data de notificação", labels={'num_casos':'N° de casos ativos', 'data_notificacao': 'Data de notificação'})

fig.add_trace(
    go.Scatter(
        x=casos_ativos_goiania['data_notificacao'],
        y=casos_ativos_goiania['MM_casos'],
        name='Média Móvel<br>(7 dias)',        
        hovertemplate= "Média Móvel: %{x}<br>Data: %{y}<extra></extra>",

    ))

fig.update()

In [137]:
import plotly.graph_objects as go
fig.show()
fig.write_image("casos_ativos.svg")
fig.write_image("casos_ativos.png")

In [138]:
from google.colab import files
files.download('casos_ativos.svg')
files.download('casos_ativos.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Casos por dia

*   Gráfico de barras com média móvel




```
dataframe: casos_por_dia
gera: casos_por_dia_goiania.csv
```


In [21]:
casos_por_dia = casos_goiania_cidade
casos_por_dia_remove = casos_por_dia.loc[((casos_por_dia['municipio'] != 'GOIANIA'))]
# df_remove = df.loc[(df['recuperado'] != 'NAO')]

casos_por_dia = casos_por_dia.drop(casos_por_dia_remove.index)
casos_por_dia

,data_notificacao,data_inicio_sintomas,ano_epi,semana_epi,sexo,raca_cor,faixa_etaria,diabetes,doenca_cardiovascular,doenca_respiratoria,imunossupressao,codigo_ibge,municipio,regiao_saude,recuperado
10,20200805,20200728,2020,31,FEMININO,IGNORADO,>= 80 anos,NAO,NAO,SIM,NAO,520870,GOIANIA,CENTRAL,NAO
16,20200722,20200715,2020,29,FEMININO,PARDA,40 a 49 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,SIM
17,20200613,20200612,2020,24,FEMININO,BRANCA,70 a 79 anos,NAO,SIM,SIM,NAO,520870,GOIANIA,CENTRAL,SIM
23,20200825,20200803,2020,32,FEMININO,PARDA,40 a 49 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,SIM
25,20200828,20200827,2020,35,FEMININO,PRETA,15 a 19 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183710,20200916,20200818,2020,34,MASCULINO,PARDA,< 10 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,SIM
183711,20200806,20200801,2020,31,MASCULINO,PARDA,20 a 29 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,SIM
183712,20200708,20200624,2020,26,MASCULINO,PARDA,20 a 29 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,SIM
183714,20200821,20200811,2020,33,MASCULINO,PARDA,20 a 29 anos,NAO,NAO,NAO,NAO,520870,GOIANIA,CENTRAL,SIM


In [22]:
casos_por_dia = casos_por_dia['data_notificacao'].value_counts() 
casos_por_dia = casos_por_dia.to_frame(name='num_casos')
casos_por_dia.reset_index(level=0, inplace=True)
casos_por_dia.rename(columns={'index': 'data_notificacao'}, inplace = True)
casos_por_dia

,data_notificacao,num_casos
0,20200812,1195
1,20200811,1074
2,20200817,1041
3,20200813,996
4,20200804,977
...,...,...
187,20200314,1
188,20200919,1
189,20200313,1
190,20200311,1


In [23]:
casos_por_dia['data_notificacao'] = casos_por_dia['data_notificacao'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d').date())

casos_por_dia


,data_notificacao,num_casos
0,2020-08-12,1195
1,2020-08-11,1074
2,2020-08-17,1041
3,2020-08-13,996
4,2020-08-04,977
...,...,...
187,2020-03-14,1
188,2020-09-19,1
189,2020-03-13,1
190,2020-03-11,1


In [24]:
casos_por_dia['data_notificacao'] = pd.to_datetime(casos_por_dia['data_notificacao'])

# Ordena pela data e salva em um novo DataFrame
casos_por_dia = casos_por_dia.sort_values(by='data_notificacao')
casos_por_dia

,data_notificacao,num_casos
191,2020-03-01,1
186,2020-03-10,1
190,2020-03-11,1
189,2020-03-13,1
187,2020-03-14,1
...,...,...
6,2020-09-15,943
41,2020-09-16,467
53,2020-09-17,357
61,2020-09-18,294


In [25]:
casos_por_dia.to_csv('casos_por_dia_goiania.csv', index=False)
casos_por_dia

,data_notificacao,num_casos
191,2020-03-01,1
186,2020-03-10,1
190,2020-03-11,1
189,2020-03-13,1
187,2020-03-14,1
...,...,...
6,2020-09-15,943
41,2020-09-16,467
53,2020-09-17,357
61,2020-09-18,294


### Média móvel - casos confirmados por dia

In [26]:
# casos_ativos_goiania = pd.read_csv('casos_ativos_goiania.csv', sep=',')

In [27]:
df = casos_por_dia
for i in range(0,df.shape[0]-6):
    df.loc[df.index[i+6],'MM_casos'] = np.round(((df.iloc[i,1]+
                                                  df.iloc[i+1,1]+
                                                  df.iloc[i+2,1]+
                                                  df.iloc[i+3,1]+
                                                  df.iloc[i+4,1]+
                                                  df.iloc[i+5,1]+
                                                  df.iloc[i+6,1])/7),1)           

df.head(8)

,data_notificacao,num_casos,MM_casos
191,2020-03-01,1,NaN
186,2020-03-10,1,NaN
190,2020-03-11,1,NaN
189,2020-03-13,1,NaN
187,2020-03-14,1,NaN
177,2020-03-15,2,NaN
176,2020-03-16,4,1.6
184,2020-03-17,1,1.6


In [28]:
casos_por_dia.to_csv('casos_ativos_goiania.csv', index=False)

In [124]:
fig = px.bar(casos_por_dia, x='data_notificacao', y='num_casos',  title="Casos confirmados por data de notificação", labels={'num_casos':'N° de casos confirmados', 
                                                                         'data_notificacao': 'Data de notificação',})

fig.add_trace(
    go.Scatter(
        x=casos_por_dia['data_notificacao'],
        y=casos_por_dia['MM_casos'],
        name='Média Móvel<br>(7 dias)',        
        hovertemplate= "Média Móvel: %{x}<br>Data: %{y}<extra></extra>",

    ))

fig.update_traces()

In [125]:
import plotly.graph_objects as go
fig.show()
fig.write_image("casos_por_dia.svg")
fig.write_image("casos_por_dia.png")

In [126]:
from google.colab import files
files.download('casos_por_dia.svg')
files.download('casos_por_dia.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Casos acumulados por dia

*   Gráfico de barras com média móvel




```
dataframe: casos_goiania_cidade
gera: casos_acumulados_por_dia_goiania.csv
```


In [30]:
casos_acumulados = pd.read_csv('casos_por_dia_goiania.csv', sep=',')
casos_acumulados = casos_acumulados.drop(columns=['MM_casos'])
casos_acumulados

KeyError: ignored

In [31]:
casos_acumulados['acumulado'] = casos_acumulados['num_casos'].cumsum()
casos_acumulados

,data_notificacao,num_casos,acumulado
0,2020-03-01,1,1
1,2020-03-10,1,2
2,2020-03-11,1,3
3,2020-03-13,1,4
4,2020-03-14,1,5
...,...,...,...
187,2020-09-15,943,46547
188,2020-09-16,467,47014
189,2020-09-17,357,47371
190,2020-09-18,294,47665


In [32]:
casos_acumulados.to_csv('casos_acumulados_por_dia_goiania.csv', index=False)


### Média móvel - casos confirmados por dia

In [33]:
# casos_ativos_goiania = pd.read_csv('casos_ativos_goiania.csv', sep=',')

In [34]:
fig = px.bar(casos_acumulados, x='data_notificacao', y='acumulado', labels={'acumulado':'N° de casos confirmados', 
                                                                         'data_notificacao': 'Data de notificação',})


fig.update_traces()